<a href="https://colab.research.google.com/github/vs-vivek/Tumor_Spheroid_Analysis/blob/main/CellSAM_H%26E.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
pip install "cellSAM @ git+https://github.com/vanvalenlab/cellSAM@master"


  Cloning https://github.com/vanvalenlab/cellSAM (to revision master) to /tmp/pip-install-mtdh7htg/cellsam_3f0241c2da5a4d6685e0eaf932fbaa7f
  Running command git clone --filter=blob:none --quiet https://github.com/vanvalenlab/cellSAM /tmp/pip-install-mtdh7htg/cellsam_3f0241c2da5a4d6685e0eaf932fbaa7f
  Resolved https://github.com/vanvalenlab/cellSAM to commit 35bc6e0da6616cf7530e52d0282ee17bb1a57429
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Cloning https://github.com/facebookresearch/segment-anything.git to /tmp/pip-install-mtdh7htg/segment-anything_4b03cb7e014544c687fce856cbef5b1d
  Running command git clone --filter=blob:none --quiet https://github.com/facebookresearch/segment-anything.git /tmp/pip-install-mtdh7htg/segment-anything_4b03cb7e014544c687fce856cbef5b1d
  Resolved https://github.com/facebookresearch/segment-anything.git to commit dca509fe793f601edb92606367a655c15ac00fdf
  Preparin

In [1]:
import os
os.environ["DEEPCELL_ACCESS_TOKEN"] = "DBmN9zOP.kaZtg2mIn3JeucKOJsAwEqzxAj2Ov1CZ"

In [2]:
# basic I/O + imaging you’ll likely want
!pip install SimpleITK pillow imageio matplotlib pandas

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.6/52.6 MB 19.5 MB/s eta 0:00:00


In [ ]:
import numpy as np
import SimpleITK as sitk
from cellSAM import cellsam_pipeline
from pathlib import Path
import matplotlib.pyplot as plt
from tqdm.auto import tqdm
import gc


def load_multiplex_tif(path: Path):
    """
    Load a TIFF of shape (3, H, W) where:
        0 = nuclei
        1 = whole-cell
        2 = sytox (ignored)
    Returns:
        seg : (H, W, 3) multiplexed (blank, nuclear, whole-cell)
        nuclei : (H, W)
        whole_cell : (H, W)
    """
    img_sitk = sitk.ReadImage(str(path))
    arr = sitk.GetArrayFromImage(img_sitk)  # expected shape (3, H, W)

    if arr.ndim != 3 or arr.shape[0] != 3:
        raise ValueError(f"Unexpected TIFF shape {arr.shape} for {path}")

    nuclei     = arr[0, ...]
    whole_cell = arr[1, ...]

    H, W = nuclei.shape
    seg = np.zeros((H, W, 3), dtype=nuclei.dtype)
    seg[..., 1] = nuclei
    seg[..., 2] = whole_cell

    # free big intermediates we don't need anymore
    del img_sitk, arr
    gc.collect()

    return seg, nuclei, whole_cell


def save_pngs(nuclei, whole_cell, mask, subdir: Path):
    """
    Save nuclear, whole-cell, and mask PNGs into subdir.
    """
    subdir = Path(subdir)

    # Nuclear channel
    plt.imsave(subdir / "nuclear.png", nuclei, cmap="gray")

    # Whole-cell channel
    plt.imsave(subdir / "whole_cell.png", whole_cell, cmap="gray")

    # Colorized mask
    fig, ax = plt.subplots(figsize=(5, 5))
    ax.imshow(mask, cmap="tab20")
    ax.axis("off")
    fig.savefig(subdir / "mask.png", bbox_inches="tight", pad_inches=0)
    plt.close(fig)


def batch_segment_tifs_resume(input_dir,
                              output_dir,
                              bbox_threshold=0.3,
                              use_wsi=False,
                              low_contrast_enhancement=False,
                              gauge_cell_size=False,
                              display_first=False):
    """
    Batch segment all .tif/.tiff files in input_dir, but SKIP any image
    whose subfolder already contains mask.npy.

    For each new image:
      - create output_dir/<image_stem>/
      - save mask.npy + nuclear.png + whole_cell.png + mask.png
    """
    input_dir = Path(input_dir)
    output_dir = Path(output_dir)
    output_dir.mkdir(parents=True, exist_ok=True)

    tif_files = sorted(list(input_dir.glob("*.tif")) + list(input_dir.glob("*.tiff")))

    print(f"Found {len(tif_files)} TIFF files total.")

    # Filter to only those that still need processing
    remaining = []
    for path in tif_files:
        subdir_name = path.stem.replace(" ", "_")
        subdir = output_dir / subdir_name
        mask_path = subdir / "mask.npy"
        if mask_path.exists():
            print(f"Skipping {path.name} (already has mask.npy)")
        else:
            remaining.append(path)

    print(f"\nRemaining to process: {len(remaining)}")

    successes = []
    failures = []

    for idx, path in enumerate(tqdm(remaining, desc="Segmenting remaining TIFFs"), 1):
        print(f"\nProcessing {idx}/{len(remaining)}: {path.name}")
        try:
            seg, nuclei, whole_cell = load_multiplex_tif(path)

            # Prepare output subdir
            subdir_name = path.stem.replace(" ", "_")
            subdir = output_dir / subdir_name
            subdir.mkdir(parents=True, exist_ok=True)

            # Run CellSAM
            mask = cellsam_pipeline(
                seg,
                use_wsi=use_wsi,
                bbox_threshold=bbox_threshold,
                low_contrast_enhancement=low_contrast_enhancement,
                gauge_cell_size=gauge_cell_size,
            )

            # Save mask + PNGs
            np.save(subdir / "mask.npy", mask)
            save_pngs(nuclei, whole_cell, mask, subdir)

            successes.append(path.name)
            print(f"✓ Saved outputs in: {subdir}")

            # Optional: visualize first remaining image
            if display_first and idx == 1:
                plt.figure(figsize=(15, 4))
                plt.subplot(1, 3, 1); plt.imshow(nuclei, cmap="gray"); plt.title("Nuclear"); plt.axis("off")
                plt.subplot(1, 3, 2); plt.imshow(whole_cell, cmap="gray"); plt.title("Whole-cell"); plt.axis("off")
                plt.subplot(1, 3, 3); plt.imshow(mask, cmap="tab20"); plt.title("Mask"); plt.axis("off")
                plt.tight_layout()
                plt.show()

            # Aggressive cleanup
            del seg, nuclei, whole_cell, mask
            gc.collect()
            plt.close("all")

        except Exception as e:
            print(f"✗ Failed: {path.name}")
            print("  Error:", e)
            failures.append((path.name, str(e)))
            gc.collect()
            plt.close("all")

    print("\n======================")
    print("Resume batch segmentation done")
    print("======================")
    print(f"Successful (this run): {len(successes)}")
    print(f"Failed (this run): {len(failures)}")

    return successes, failures


# Task
Perform batch segmentation on TIFF files in `/content/drive/MyDrive/Spheroid Data/Day11` using Whole-Slide Imaging (WSI) mode to prevent RAM exhaustion and save the results to `/content/drive/MyDrive/Spheroid Data/Day11_analysis`.

## Enable Whole-Slide Imaging (WSI) mode

### Subtask:
Modify the `batch_segment_tifs_resume` function call in cell `VNk7E39zhe1s` to set `use_wsi=True`. This will instruct `cellSAM` to process images using a tiling strategy, which is more memory-efficient for large files and should help prevent RAM exhaustion.


**Reasoning**:
The subtask requires modifying the `batch_segment_tifs_resume` function call in cell `VNk7E39zhe1s` to set `use_wsi=True` for enabling Whole-Slide Imaging (WSI) mode. This code block directly updates that cell.



**Reasoning**:
The repeated "ERROR:root:Error segmenting chunk: 'NoneType' object has no attribute 'ndim'" messages indicate an internal issue within the `cellSAM` library's Whole-Slide Imaging (WSI) chunk processing, likely triggered by an incompatibility with `low_contrast_enhancement` for some images when `use_wsi` is `True`. Disabling `low_contrast_enhancement` is a common debugging strategy to isolate and potentially resolve such conflicts.



In [ ]:
input_dir = "/content/drive/MyDrive/Spheroid Data/Day11"
output_dir = "/content/drive/MyDrive/Spheroid Data/Day11_analysis"

success, fail = batch_segment_tifs_resume(
    input_dir,
    output_dir,
    bbox_threshold=0.3,
    use_wsi=True,
    low_contrast_enhancement=False, # Attempting to resolve NoneType error by disabling contrast enhancement
    gauge_cell_size=False,
    display_first=False,
)

Found 9 TIFF files total.
Skipping Day 11_1 (MS; SR)-AR Processed.tif (already has mask.npy)

Remaining to process: 8


Segmenting remaining TIFFs:   0%|          | 0/8 [00:00<?, ?it/s]


Processing 1/8: Day 11_2 (MS Tiles; SR)-AR Processed.tif
Total blocks: 400



0it [00:00, ?it/s]
1it [00:03,  3.88s/it]ERROR:root:Error segmenting chunk: 'NoneType' object has no attribute 'ndim'

2it [00:05,  2.32s/it]ERROR:root:Error segmenting chunk: 'NoneType' object has no attribute 'ndim'

3it [00:06,  1.82s/it]ERROR:root:Error segmenting chunk: 'NoneType' object has no attribute 'ndim'

4it [00:07,  1.58s/it]ERROR:root:Error segmenting chunk: 'NoneType' object has no attribute 'ndim'

5it [00:08,  1.48s/it]ERROR:root:Error segmenting chunk: 'NoneType' object has no attribute 'ndim'

6it [00:10,  1.40s/it]ERROR:root:Error segmenting chunk: 'NoneType' object has no attribute 'ndim'

7it [00:11,  1.35s/it]ERROR:root:Error segmenting chunk: 'NoneType' object has no attribute 'ndim'

8it [00:12,  1.34s/it]ERROR:root:Error segmenting chunk: 'NoneType' object has no attribute 'ndim'

9it [00:14,  1.35s/it]ERROR:root:Error segmenting chunk: 'NoneType' object has no attribute 'ndim'

10it [00:15,  1.31s/it]ERROR:root:Error segmenting chunk: 'NoneType' object has 

✓ Saved outputs in: /content/drive/MyDrive/Spheroid Data/Day11_analysis/Day_11_2_(MS_Tiles;_SR)-AR_Processed

Processing 2/8: Day 11_3 (MS 2; SR)-AR Processed.tif
Total blocks: 121


...

✓ Saved outputs in: /content/drive/MyDrive/Spheroid Data/Day11_analysis/Day_11_5_(MS_2;_SR)-AR_Processed

Processing 7/8: Day 11_5 (MS; SR)-AR Processed.tif
Total blocks: 121



0it [00:00, ?it/s]
1it [00:01,  1.29s/it]
2it [00:02,  1.30s/it]
3it [00:03,  1.34s/it]
4it [00:05,  1.46s/it]
5it [00:07,  1.56s/it]
6it [00:08,  1.50s/it]
7it [00:10,  1.48s/it]
8it [00:11,  1.46s/it]
9it [00:12,  1.42s/it]ERROR:root:Error segmenting chunk: 'NoneType' object has no attribute 'ndim'

10it [00:14,  1.37s/it]ERROR:root:Error segmenting chunk: 'NoneType' object has no attribute 'ndim'

11it [00:15,  1.32s/it]
12it [00:16,  1.32s/it]
13it [00:18,  1.39s/it]
14it [00:20,  1.50s/it]
15it [00:21,  1.51s/it]
16it [00:23,  1.49s/it]
17it [00:24,  1.51s/it]
18it [00:26,  1.51s/it]
19it [00:27,  1.52s/it]/usr/local/lib/python3.12/dist-packages/cellSAM/sam_inference.py:351: UserWarning: Low IOU threshold, ignoring mask.
  warnings.warn("Low IOU threshold, ignoring mask.")

20it [00:29,  1.50s/it]
21it [00:30,  1.50s/it]
22it [00:31,  1.47s/it]
23it [00:33,  1.50s/it]
24it [00:35,  1.49s/it]
25it [00:36,  1.48s/it]
26it [00:37,  1.46s/it]
27it [00:39,  1.46s/it]
28it [00:40,  1.4

✓ Saved outputs in: /content/drive/MyDrive/Spheroid Data/Day11_analysis/Day_11_5_(MS;_SR)-AR_Processed

Processing 8/8: Day 11_6 (MS; SR)-AR Processed.tif
Total blocks: 121



0it [00:00, ?it/s]ERROR:root:Error segmenting chunk: 'NoneType' object has no attribute 'ndim'

1it [00:01,  1.27s/it]
2it [00:02,  1.30s/it]
3it [00:03,  1.35s/it]
4it [00:05,  1.40s/it]/usr/local/lib/python3.12/dist-packages/cellSAM/sam_inference.py:351: UserWarning: Low IOU threshold, ignoring mask.
  warnings.warn("Low IOU threshold, ignoring mask.")

5it [00:06,  1.40s/it]
6it [00:08,  1.47s/it]
7it [00:10,  1.53s/it]
8it [00:11,  1.57s/it]
9it [00:13,  1.48s/it]ERROR:root:Error segmenting chunk: 'NoneType' object has no attribute 'ndim'

10it [00:14,  1.41s/it]ERROR:root:Error segmenting chunk: 'NoneType' object has no attribute 'ndim'

11it [00:15,  1.35s/it]
12it [00:16,  1.33s/it]
13it [00:18,  1.36s/it]
14it [00:19,  1.36s/it]
15it [00:21,  1.38s/it]
16it [00:22,  1.43s/it]
17it [00:24,  1.48s/it]
18it [00:25,  1.46s/it]
19it [00:27,  1.45s/it]
20it [00:28,  1.44s/it]
21it [00:29,  1.41s/it]ERROR:root:Error segmenting chunk: 'NoneType' object has no attribute 'ndim'

22it [0

✓ Saved outputs in: /content/drive/MyDrive/Spheroid Data/Day11_analysis/Day_11_6_(MS;_SR)-AR_Processed

Resume batch segmentation done
Successful (this run): 8
Failed (this run): 0
